In [1]:
import pyspark.sql.functions as F
from pyspark.sql.types import StructType, StructField, StringType, FloatType

In [2]:
from pyspark.sql import SparkSession

In [3]:
from delta import *

In [4]:
spark = SparkSession.builder \
    .appName("pyspark-sample") \
    .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension") \
    .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog") \
    .config("spark.executor.instances", "1") \
    .config("spark.executor.memory", "1g") \
    .getOrCreate()

/opt/spark/bin/load-spark-env.sh: line 68: ps: command not found


:: loading settings :: url = jar:file:/opt/spark/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /root/.ivy2/cache
The jars for the packages stored in: /root/.ivy2/jars
software.amazon.awssdk#s3 added as a dependency
org.apache.hadoop#hadoop-aws added as a dependency
io.delta#delta-spark_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-658a86ab-ffce-4128-849e-2ec84eaa048e;1.0
	confs: [default]
	found software.amazon.awssdk#s3;2.26.30 in central
	found software.amazon.awssdk#aws-xml-protocol;2.26.30 in central
	found software.amazon.awssdk#aws-query-protocol;2.26.30 in central
	found software.amazon.awssdk#protocol-core;2.26.30 in central
	found software.amazon.awssdk#sdk-core;2.26.30 in central
	found software.amazon.awssdk#annotations;2.26.30 in central
	found software.amazon.awssdk#http-client-spi;2.26.30 in central
	found software.amazon.awssdk#utils;2.26.30 in central
	found org.reactivestreams#reactive-streams;1.0.4 in central
	found org.slf4j#slf4j-api;1.7.36 in central
	found software.amazon.awssdk#metric

In [5]:
df = spark.range(0, 100_000_000)
df = df.withColumn("content", F.lit("Anything"))

In [6]:
(
    df.write
    .format("parquet")
    .mode("overwrite")
    .save("s3a://raw/sample")
)

In [7]:
(
    df.write
    .format("delta")
    .mode("overwrite")
    .save("s3a://bronze/sample")
)